# ReAct logic test notebook

Цей ноутбук допомагає швидко перевірити підтримку structured output у LLM
та прогнати `react_loop_node` на простому прикладі.

Перед запуском переконайтеся, що vLLM/OpenAI endpoint і Graphiti доступні
та налаштовані через `config.settings` або змінні оточення.

In [1]:
from pathlib import Path
import sys
import asyncio

from pydantic import BaseModel

repo_root = Path("../graphity_lapa")
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from clients.llm_client import get_llm_client
from agent.nodes.react import react_loop_node, ReactStep
from agent.state import create_initial_state

In [4]:
# Перевірка structured output
llm = get_llm_client()

class SimpleStep(BaseModel):
    thought: str
    action: str
    query: str

prompt = """Поверни JSON з ключами: thought, action, query.
thought: 'test', action: 'answer', query: ''
"""

try:
    result = await llm.generate_async(
        messages=[{"role": "user", "content": prompt}],
        # response_format=SimpleStep,
        temperature=0.1,
        max_tokens=50,
        timeout=2
    )
    print(result)
    print(type(result))
except Exception as exc:
    print("Structured output не підтримується або виникла помилка:", exc)

Error generating response: Connection error.


Structured output не підтримується або виникла помилка: Connection error.


In [ ]:
# Прогін react_loop_node
state = create_initial_state(
    message_uid="demo-1",
    message_text="Поясни, що таке ReAct агент простими словами",
    user_id="local_test",
)

# Приклад додаткового контексту (якщо потрібно)
state["retrieved_context"] = [
    {
        "source_msg_uid": "seed-1",
        "content": "ReAct поєднує reasoning і acting, чергуючи думки та дії.",
    }
]

result = await react_loop_node(state)
print(result)